# <center>Adaptive Filters</center>
***
<center>Natanael Magno Gomes <b>397645</b></center>

***
## Kalman Filter assignment

For this assignment, consider an old Volkswagen Beetle that is driving on a long straight road. Using a Kalman filter, we want to estimate the Beetle’s position on the road and the speed at which it is travelling. First based on speed measurements and then based on position measurements.<br>
1. Assume that (in theory) the Beetle is travelling at a speed of 90 km/h. 
Simulate velocity measurements by adding white noise (mean = 0 km/h, standard deviation = 20 km/h) to the theoretical speed and set the sample frequency to one sample per minute.<br> Design and build a Kalman filter (for example in Python or Octave/Matlab) to estimate the Beetle’s velocity and use the velocity estimates to derive its (one-dimensional) position on the road. (Assume at time 0 the Beetle is at km 0 along the road.) Present your results graphically and comment on them.
2. The Beetle’s owner recently bought a navigation system that uses GPS positioning. Assume again that the Beetle is travelling at a speed of 90 km/h and determine the theoretical positions of the car along the road at 1-minute intervals.<br>The road is passing through a forest, so the GPS measurements are rather noisy. Suppose that the GPS measurements give you the Beetle’s positions with additive white noise (mean = 0 m, standard deviation = 300 m) once per minute.<br>Design and build a Kalman filter to first estimate the Beetle’s position and then use the estimated positions to derive its velocity. (Again assume at time 0 the car is at km 0 along the road.)<br>Present your results graphically and comment on them.<br>Compare the new results with the results from (1) and comment on the differences (if any). 


